<a href="https://colab.research.google.com/github/adadoun/KplerNextDestination/blob/main/NNDestinationProbabilityPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vessel Destination Prediction

## 1. Import Libraries

In [2]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.6 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from category_encoders import TargetEncoder
import lightgbm as lgb
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

## 2. Data Preprocessing

### 2.1 Define Preprocessing Function


In [5]:
# Preprocessing function
def preprocess_data(df, is_train=True, cat_encodings=None, scaler=None):
    cat_cols = ['vessel_id',
                'origin', 'destination',
                'origin_h3_res2_index', 'destination_h3_res2_index',
                'product_family', 'vessel_type',
                'products',
               'flag_name', 'origin_country_code', 'destination_country_code',
               'previous_visited_port_1', 'previous_visited_port_2', 'previous_visited_port_3',
               'probability_level'
               ]
    num_cols = ['day_of_week', 'month', 'traded_volume',
               'dead_weight', 'vessel_age', 'origin_draught_change', 'destination_draught_change',
                'origin_cargo_volume', 'destination_cargo_volume', 'od_distance', 'p1_destination_probability',
                'p2_destination_probability', 'p3_destination_probability',
                'p4_destination_probability', 'merged_destination_probability'
               ]

    if is_train:
        cat_encodings = {col: defaultdict(lambda: 0, {val: i + 1 for i, val in enumerate(df[col].unique())}) for col in cat_cols}
        for col in cat_encodings:
            cat_encodings[col]['<unknown>'] = len(cat_encodings[col])
        scaler = StandardScaler()
        df[num_cols] = scaler.fit_transform(df[num_cols].fillna(0))
    else:
        df[num_cols] = scaler.transform(df[num_cols].fillna(0))

    for col in cat_cols:
        df[col] = df[col].map(lambda x: cat_encodings[col][x] if x in cat_encodings[col] else cat_encodings[col]['<unknown>'])

    return df, cat_encodings, scaler, cat_cols, num_cols

# Dataset class
class VesselDataset(Dataset):
    def __init__(self, features, labels=None):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        if self.labels is not None:
            return torch.tensor(self.features.iloc[idx].values, dtype=torch.float), torch.tensor(self.labels.iloc[idx], dtype=torch.float)
        return torch.tensor(self.features.iloc[idx].values, dtype=torch.float)

# Model definition
class VesselDestinationModel(nn.Module):
    def __init__(self, cat_dims, num_dims, embedding_dims=[128,
                                                           128, 128,
                                                           64, 64,
                                                           10, 10, 10, 10, 10, 10,
                                                           128, 128, 128,
                                                           1],
                                             hidden_dims=[1024, 512, 256, 128, 64, 32]):
        super(VesselDestinationModel, self).__init__()

        self.embeddings = nn.ModuleList([nn.Embedding(dim, embedding_dims[i]) for i, dim in enumerate(cat_dims)])

        total_embedding_dim = sum(embedding_dims)
        self.num_features = num_dims
        total_input_dim = total_embedding_dim + self.num_features

        layers = []
        input_dim = total_input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.ReLU())
            input_dim = hidden_dim

        layers.append(nn.Linear(hidden_dims[-1], 1))

        self.main_network = nn.Sequential(*layers)


        # The element-wise products will result in two scalar values
        self.final_layer = nn.Linear(3, 1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        cat_features = x[:, :len(self.embeddings)].long()
        num_features = x[:, len(self.embeddings):]

        embedded = [emb(cat_features[:, i]) for i, emb in enumerate(self.embeddings)]

        # Extract specific embeddings
        vessel_id_emb = embedded[0]
        origin_emb = embedded[1]
        destination_emb = embedded[2]

        # Compute element-wise products
        origin_dest_product = (origin_emb * destination_emb).sum(dim=1, keepdim=True)
        vessel_dest_product = (vessel_id_emb * destination_emb).sum(dim=1, keepdim=True)

        # Concatenate all embeddings
        embedded = torch.cat(embedded, dim=1)

        # Concatenate embeddings with numerical features
        x = torch.cat([embedded, num_features], dim=1)

        # Pass through main network
        main_output = self.main_network(x)

        # Concatenate main output with element-wise products
        final_input = torch.cat([main_output, origin_dest_product, vessel_dest_product], dim=1)

        # Final layer and activation
        output = self.activation(self.final_layer(final_input))

        return output.squeeze()


# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0

        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            predicted = (outputs > 0.5).float()
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_loss /= len(train_loader)
        train_accuracy = 100 * train_correct / train_total

        # Validation
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                predicted = (outputs > 0.5).float()
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = 100 * val_correct / val_total

        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')
        print()

print("Libraries imported and functions defined successfully.")

Libraries imported and functions defined successfully.


In [6]:
# Load and preprocess training data
train_df = pd.read_csv('drive/MyDrive/Collab_DATA/train_trades_csv_prepared.csv')
train_df, cat_encodings, scaler, cat_cols, num_cols = preprocess_data(train_df, is_train=True)

# Split into train and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# Split the data
# Sort the dataframe by end_date_time
df_sorted = train_df.sort_values('start_date_time')

# Determine the split point (90% train, 10% validation)
split_index = int(len(df_sorted) * 0.95)

# Split the data
train_df = df_sorted.iloc[:split_index]
val_df = df_sorted.iloc[split_index:]



print("Training data preprocessed and split.")
print(f"Shape of training data: {train_data.shape}")
print(f"Shape of validation data: {val_data.shape}")
print(f"Training data date range: {train_df['start_date_time'].min()} to {train_df['start_date_time'].max()}")
print(f"Validation data date range: {val_df['start_date_time'].min()} to {val_df['start_date_time'].max()}")


# Create datasets and dataloaders
train_dataset = VesselDataset(train_data[cat_cols + num_cols], train_data['is_visit'])
val_dataset = VesselDataset(val_data[cat_cols + num_cols], val_data['is_visit'])
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256)

print("Datasets and DataLoaders created.")
print(f"Number of batches in train_loader: {len(train_loader)}")
print(f"Number of batches in val_loader: {len(val_loader)}")


Training data preprocessed and split.
Shape of training data: (476392, 33)
Shape of validation data: (119098, 33)
Training data date range: 2023-01-01T00:35:13.000Z to 2023-10-07T04:39:12.000Z
Validation data date range: 2023-10-07T04:39:12.000Z to 2023-10-30T18:35:58.000Z
Datasets and DataLoaders created.
Number of batches in train_loader: 1861
Number of batches in val_loader: 466


In [7]:
# Initialize the model
cat_dims = [len(cat_encodings[col]) for col in cat_cols]
num_dims = len(num_cols)
model = VesselDestinationModel(cat_dims, num_dims)

# Training setup
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Model initialized. Using device: {device}")
print(model)

Model initialized. Using device: cuda
VesselDestinationModel(
  (embeddings): ModuleList(
    (0): Embedding(8394, 128)
    (1): Embedding(1343, 128)
    (2): Embedding(1549, 128)
    (3): Embedding(501, 64)
    (4): Embedding(601, 64)
    (5): Embedding(12, 10)
    (6): Embedding(13, 10)
    (7): Embedding(2299, 10)
    (8): Embedding(94, 10)
    (9): Embedding(132, 10)
    (10): Embedding(152, 10)
    (11): Embedding(1535, 128)
    (12): Embedding(1512, 128)
    (13): Embedding(1490, 128)
    (14): Embedding(6, 1)
  )
  (main_network): Sequential(
    (0): Linear(in_features=972, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=128, bias=True)
    (7): ReLU()
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): ReLU()
    (10): Linear(in_features=64, out_features=32, bia

## 3. Define PyTorch Dataset and Model

### 3.1 PyTorch Dataset

In [8]:
num_epochs = 3
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

print("Model training completed.")

Epoch 1/3:
Train Loss: 0.4442, Train Accuracy: 89.60%
Val Loss: 0.1270, Val Accuracy: 95.43%

Epoch 2/3:
Train Loss: 0.0858, Train Accuracy: 96.97%
Val Loss: 0.0830, Val Accuracy: 97.10%

Epoch 3/3:
Train Loss: 0.0473, Train Accuracy: 98.30%
Val Loss: 0.0750, Val Accuracy: 97.30%

Model training completed.


### 3.2 Model Definition

In [36]:
# Load test data
test_df = pd.read_csv('drive/MyDrive/Collab_DATA/test_trades_csv_prepared.csv')

In [38]:
import random
# Select 100 random unique vessel_ids
random_vessels = random.sample(test_df['vessel_id'].unique().tolist(), 6000)

# Filter test_df to include only the selected vessels
test_df_sample = test_df[test_df['vessel_id'].isin(random_vessels)]


In [39]:
# Preprocess the sampled test data
test_df_sample, _, _, _, _ = preprocess_data(test_df_sample, is_train=False, cat_encodings=cat_encodings, scaler=scaler)

# Create test dataset and dataloader
test_dataset = VesselDataset(test_df_sample[cat_cols + num_cols])
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

print("Test data preprocessed.")
print(f"Number of vessels in sample: {len(random_vessels)}")
print(f"Shape of preprocessed test data: {test_df_sample.shape}")

<ipython-input-5-594fc7e770b7>:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-594fc7e770b7>:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Test data preprocessed.
Number of vessels in sample: 6000
Shape of preprocessed test data: (2081521, 33)


In [40]:
model.eval()
predictions = []
with torch.no_grad():
    for features in test_loader:
        features = features.to(device)
        outputs = model(features)
        predictions.extend(outputs.cpu().numpy())

# Add predictions to test_df_sample
test_df_sample['predicted_probability'] = predictions


<ipython-input-40-8a817d8c0cb9>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
# Step 1: Prepare the data
ranked_destinations = test_df_sample[test_df_sample.is_visit == 0].groupby('vessel_id').apply(
    lambda x: x.sort_values('predicted_probability', ascending=False)
).reset_index(drop=True)

actual_destinations = test_df_sample[test_df_sample['is_visit'] == 1].set_index('vessel_id')['destination'].to_dict()


In [42]:
# Step 2: Define utility functions
def calculate_reciprocal_rank(group):
    vessel_id = group['vessel_id'].iloc[0]
    actual_dest = actual_destinations.get(vessel_id)
    if actual_dest is None:
        return 0
    actual_dest_rows = group[group['destination'] == actual_dest]
    if actual_dest_rows.empty:
        return 0
    rank = group.index.get_loc(actual_dest_rows.index[0]) + 1
    return 1 / rank

def is_in_top_n(group, n):
    reciprocal_rank = calculate_reciprocal_rank(group)
    return reciprocal_rank >= 1/n if reciprocal_rank > 0 else False


In [43]:
# Step 3: Calculate metrics
mrr = ranked_destinations.groupby('vessel_id').apply(calculate_reciprocal_rank).mean()
top_1_accuracy = ranked_destinations.groupby('vessel_id').apply(lambda x: is_in_top_n(x, 1)).mean()
top_3_accuracy = ranked_destinations.groupby('vessel_id').apply(lambda x: is_in_top_n(x, 3)).mean()
top_10_accuracy = ranked_destinations.groupby('vessel_id').apply(lambda x: is_in_top_n(x, 10)).mean()

print(f"Mean Reciprocal Rank (MRR): {mrr:.4f}")
print(f"Top 1 Accuracy: {top_1_accuracy:.4f}")
print(f"Top 3 Accuracy: {top_3_accuracy:.4f}")
print(f"Top 10 Accuracy: {top_10_accuracy:.4f}")


Mean Reciprocal Rank (MRR): 0.6547
Top 1 Accuracy: 0.5684
Top 3 Accuracy: 0.7323
Top 10 Accuracy: 0.7850


In [44]:
# Step 4: Plot accuracy evolution from top 1 to top 10
accuracies = [ranked_destinations.groupby('vessel_id').apply(lambda x: is_in_top_n(x, i)).mean() for i in range(1, 11)]

fig1 = go.Figure(data=go.Scatter(
    x=list(range(1, 11)),
    y=accuracies,
    mode='lines+markers+text',
    text=[f'{acc:.2%}' for acc in accuracies],
    textposition='top center'
))
fig1.update_layout(
    title='Accuracy Evolution: Top 1 to Top 10',
    xaxis_title='Top N Predictions',
    yaxis_title='Accuracy',
    yaxis_tickformat='.0%',
    xaxis=dict(tickmode='linear', tick0=1, dtick=1)
)
fig1.show()


In [46]:
# Step 6: Accuracy evolution by number of past trips
train_trips = train_df[train_df['is_visit'] == 1]['vessel_id'].value_counts().reset_index()
train_trips.columns = ['vessel_id', 'num_past_trips']

combined_data = pd.merge(ranked_destinations, train_trips, on='vessel_id', how='left')
combined_data = combined_data[combined_data['num_past_trips'] <= 40]

accuracy_by_past_trips = combined_data.groupby(['num_past_trips', 'vessel_id']).apply(lambda x: is_in_top_n(x, 1)).groupby('num_past_trips').mean().reset_index()
accuracy_by_past_trips.columns = ['num_past_trips', 'top_1_accuracy']

fig3 = go.Figure()

fig3.add_trace(go.Scatter(
    x=accuracy_by_past_trips['num_past_trips'],
    y=accuracy_by_past_trips['top_1_accuracy'],
    mode='lines+markers+text',
    name='Top 1 Accuracy',
    text=[f'{acc:.2%}' for acc in accuracy_by_past_trips['top_1_accuracy']],
    textposition='top center'
))

fig3.update_layout(
    title='Evolution of Top 1 Accuracy by Number of Past Trips (Up to 40 trips)',
    xaxis_title='Number of Past Trips',
    yaxis_title='Top 1 Accuracy',
    yaxis_tickformat='.0%',
    xaxis=dict(tickmode='linear', tick0=0, dtick=1)
)

fig3.show()



In [47]:
# Print additional statistics
print("\nAdditional Statistics:")
print(f"Total number of vessels: {len(ranked_destinations['vessel_id'].unique())}")
print(f"Average number of potential destinations per vessel: {ranked_destinations['num_potential_destinations'].mean():.2f}")
print(f"Median number of potential destinations per vessel: {ranked_destinations['num_potential_destinations'].median():.2f}")
print(f"Range of potential destinations: {ranked_destinations['num_potential_destinations'].min()} to {ranked_destinations['num_potential_destinations'].max()}")
print(f"Correlation between number of potential destinations and accuracy: {accuracy_by_bin['bin_midpoint'].corr(accuracy_by_bin['mean']):.4f}")

# Statistics for vessels with more than 1000 potential destinations
vessels_over_1000 = ranked_destinations[ranked_destinations['num_potential_destinations'] > 1000]
print(f"\nNumber of vessels with more than 1000 potential destinations: {len(vessels_over_1000['vessel_id'].unique())}")
if not vessels_over_1000.empty:
    print(f"Average accuracy for vessels with >1000 potential destinations: {vessels_over_1000['top_1_accuracy'].mean():.2%}")

# Correlation between past trips and accuracy
correlation = accuracy_by_past_trips['num_past_trips'].corr(accuracy_by_past_trips['top_1_accuracy'])
print(f"\nCorrelation between number of past trips and accuracy: {correlation:.4f}")



Additional Statistics:
Total number of vessels: 5939
Average number of potential destinations per vessel: 448.43
Median number of potential destinations per vessel: 450.00
Range of potential destinations: 19 to 1142
Correlation between number of potential destinations and accuracy: 0.3524

Number of vessels with more than 1000 potential destinations: 9
Average accuracy for vessels with >1000 potential destinations: nan%

Correlation between number of past trips and accuracy: 0.5191


In [48]:
def get_original_labels(encoded_values, encoding_dict):
    return {v: k for k, v in encoding_dict.items()}

vessel_type_labels = get_original_labels(cat_encodings['vessel_type'], cat_encodings['vessel_type'])
# Pass the encoding dictionary for product family
product_family_labels = get_original_labels(cat_encodings['product_family'], cat_encodings['product_family'])


def compute_top_1_accuracy_by_dimension(dimension, top_n, label_dict):
    top_categories = ranked_destinations[dimension].value_counts().nlargest(top_n).index
    accuracies = {}
    for category in top_categories:
        subset = ranked_destinations[ranked_destinations[dimension] == category]
        accuracy = subset.groupby('vessel_id').apply(lambda x: is_in_top_n(x, 1)).mean()
        label = label_dict.get(category, f"Unknown ({category})")
        accuracies[label] = accuracy
    return pd.Series(accuracies)

vessel_type_accuracy = compute_top_1_accuracy_by_dimension('vessel_type', 4, vessel_type_labels)
product_family_accuracy = compute_top_1_accuracy_by_dimension('product_family', 5, product_family_labels)

fig = make_subplots(rows=2, cols=1,
                    subplot_titles=('Top 1 Accuracy by Vessel Type', 'Top 1 Accuracy by Product Family'),
                    vertical_spacing=0.3)  # Increase vertical spacing between subplots

fig.add_trace(go.Bar(
    x=vessel_type_accuracy.index,
    y=vessel_type_accuracy.values,
    text=[f'{acc:.2%}' for acc in vessel_type_accuracy.values],
    textposition='auto',
    marker_color='royalblue'
), row=1, col=1)

fig.add_trace(go.Bar(
    x=product_family_accuracy.index,
    y=product_family_accuracy.values,
    text=[f'{acc:.2%}' for acc in product_family_accuracy.values],
    textposition='auto',
    marker_color='lightsalmon'
), row=2, col=1)

fig.update_layout(
    height=1000,  # Increase overall height of the figure
    title_text="Top 1 Accuracy by Vessel Type and Product Family",
    showlegend=False
)

fig.update_xaxes(title_text="Vessel Type", row=1, col=1, tickangle=45)
fig.update_xaxes(title_text="Product Family", row=2, col=1, tickangle=45)
fig.update_yaxes(title_text="Top 1 Accuracy", tickformat='.0%', range=[0, 1], row=1, col=1)
fig.update_yaxes(title_text="Top 1 Accuracy", tickformat='.0%', range=[0, 1], row=2, col=1)

# Ensure y-axis starts at 0 and ends at 100%
fig.update_yaxes(range=[0, 1])

fig.show()

# Print additional statistics
print("\nTop 1 Accuracy by Vessel Type:")
print(vessel_type_accuracy)
print("\nTop 1 Accuracy by Product Family:")
print(product_family_accuracy)

# Calculate overall statistics
print("\nOverall Statistics:")
print(f"Average Top 1 Accuracy across Vessel Types: {vessel_type_accuracy.mean():.2%}")
print(f"Average Top 1 Accuracy across Product Families: {product_family_accuracy.mean():.2%}")
print(f"Vessel Type with highest accuracy: {vessel_type_accuracy.idxmax()} ({vessel_type_accuracy.max():.2%})")
print(f"Product Family with highest accuracy: {product_family_accuracy.idxmax()} ({product_family_accuracy.max():.2%})")


Top 1 Accuracy by Vessel Type:
Crude/Oil Products Tanker       0.574630
Chemical/Oil Products Tanker    0.532331
Products Tanker                 0.569429
Crude Oil Tanker                0.607362
dtype: float64

Top 1 Accuracy by Product Family:
clean petroleum products    0.543644
chem/bio                    0.556028
crude oil/condensate        0.635457
dirty petroleum products    0.556845
minor bulks                 0.500000
dtype: float64

Overall Statistics:
Average Top 1 Accuracy across Vessel Types: 57.09%
Average Top 1 Accuracy across Product Families: 55.84%
Vessel Type with highest accuracy: Crude Oil Tanker (60.74%)
Product Family with highest accuracy: crude oil/condensate (63.55%)
